<a href="https://colab.research.google.com/github/kdlogan19/CAP6610/blob/master/Homework2_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
df = pd.read_csv("ionosphere.data",header = None)

In [289]:
df_train = df.iloc[:300,:34]
df_test = df.iloc[300:,:34]
df_test.shape

(51, 34)

In [0]:
y_train = df.iloc[:300,34]

In [291]:
y_train

0      g
1      b
2      g
3      b
4      g
      ..
295    g
296    g
297    g
298    g
299    g
Name: 34, Length: 300, dtype: object

In [0]:
for i in range(len(y_train)):
  if y_train[i] =='g':
    y_train[i] = -1
  else:
    y_train[i] = 1

In [293]:
y_train

0      -1
1       1
2      -1
3       1
4      -1
       ..
295    -1
296    -1
297    -1
298    -1
299    -1
Name: 34, Length: 300, dtype: object

In [0]:
y_test = df.iloc[300:,34]


In [0]:
for j in range(300, 300+len(y_test)):
  if y_test[j] == 'g':
    y_test[j] = -1
  else:
    y_test[j] = 1

In [296]:
df_train.shape

(300, 34)

In [0]:
df_train = df_train.to_numpy()
df_test = df_test.to_numpy()
y_train = y_train.to_numpy().reshape(300,1)
y_test =  y_test.to_numpy().reshape(51,1).astype(int)



In [0]:
import numpy as np
ones = np.ones((300,1))

In [0]:
X_train = np.block([ones,df_train])

In [300]:
X_train.shape

(300, 35)

In [0]:
thetha =  np.matmul(np.linalg.pinv(X_train),y_train)

In [302]:
thetha.shape

(35, 1)

In [0]:
X_test = np.block([np.ones(shape = (51,1), dtype= int),df_test])

In [304]:
X_test.shape

(51, 35)

In [0]:
y_pred = np.matmul(X_test,thetha)
for i in range(len(y_pred)):
  if y_pred[i]<0:
    y_pred[i] = -1
  else:
    y_pred[i] = 1
y_pred = y_pred.astype(int)

In [310]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

1.0



---



---




#                 HINGE LOSS







In [0]:
import cvxpy as cp
w = cp.Variable((34,1))
b = cp.Variable()

In [0]:
loss_train = cp.sum(cp.pos(1- cp.multiply(y_train,(df_train@w + b))))

In [215]:
loss_train

Expression(CONVEX, NONNEGATIVE, ())

In [216]:
cp.Problem(cp.Minimize(loss_train),[]).solve(solver = cp.ECOS)

49.202899379210784

In [218]:
b.value

array(19.41918879)

In [0]:
y_pred_hinge = cp.Variable((51,1))

In [0]:
y_pred_hinge = cp.pos(df_test@w + b)

In [0]:
y_pred_hinge_round = [1 if i>0 else -1 for i in y_pred_hinge.value]

In [313]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred_hinge_round)

1.0

# LOGISTIC LOSS  = log(1 + exp(−yt))

In [0]:
w2 = cp.Variable((34,1))
b2 = cp.Variable()

In [0]:
loss_train = cp.sum(cp.logistic(-cp.multiply(y_train,(df_train@w2 + b2))))

In [261]:
cp.Problem(cp.Minimize(loss_train),[]).solve()

53.76804222339682

In [262]:
w2.value
b2.value

array(39.54653478)

In [0]:
y_pred_logistic = df_test@w2 + b2
y_pred_logistic_round = [1 if i>0 else -1 for i in y_pred_logistic.value]

In [314]:
accuracy_score(y_test,y_pred_logistic_round)

1.0